### Importo las librerias a usar

In [1]:
import pandas as pd
import numpy as np
import chardet
import  seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
%matplotlib inline

### Dataset
Link a dataset: https://www.kaggle.com/zynicide/wine-reviews

In [2]:
data = pd.read_csv('wine.csv')

### Creo un dataset con todos los datos de Argentina y dejo solo las columnas que voy  usar

In [3]:
data = data[data.country == 'Argentina']
columnas = data.columns.tolist()
columnas_innecesarias = ["Unnamed: 0", "country", "description", "designation", "taster_name", "taster_twitter_handle", "region_2"]
subconjunto_lista_filtrada = [x for x in columnas if x not in columnas_innecesarias]
data = data[subconjunto_lista_filtrada]

### Verifico si hay valores nulos

In [4]:
data.isnull().sum()

points       0
price       44
province     0
region_1    53
title        0
variety      0
winery       0
dtype: int64

In [5]:
antes_de_drop = data.shape[0]

### Elimino las filas duplicadas

In [6]:
data = data.drop_duplicates()

despues_de_drop = data.shape[0]
data = data.reset_index()

### Creo el campo creacion que va a contener el año en que se creo el vino (Se obtiene del title)

In [7]:
data["creacion"] = data.title.str.extract(r"\b(19\d\d|20[01]\d)\b", expand=False)

### Completo los vacios en creacion (el titulo no tenia año) con un 0 <br> Completo las regiones NaN con '' <br> Cambio el dato de creacion de string a int

In [8]:
data.creacion = data.creacion.fillna(value=0)
data.region_1 = data.region_1.fillna(value= '')
data.creacion = data.creacion.astype(int)

### Creo el campo categoria donde hay 3 tipos segun el puntaje del vino <br> <span style="color: red">0 <= 60 --- Detractor</span> <br> <span style="color: orange">61 <= 80 --- Neutro</span> <br> <span style="color: green">81 <= 100 --- Promotor</span>

In [9]:
for i in range(0, len(data)):
    if 0 <= data.loc[i, 'points'] <= 60:
        data.loc[i,'categoria'] = 'Detractor'
    elif 61 <= data.loc[i, 'points'] <= 80:
        data.loc[i,'categoria'] = 'Neutro'
    elif 81 <= data.loc[i, 'points'] <= 100:
        data.loc[i,'categoria'] = 'Promotor'
    else:
        data.loc[i,'categoria'] = ''

### Verifico cada una de las columnas, si el tipo es 0 (string) le limpio los carecteres especiales, le saco las tildes y los paso a minuscula

In [10]:
for x in range (0, len(data.columns)):
    if data.dtypes[x] == 'O':
        data[data.columns[x]] = data[data.columns[x]].str.lower().str.strip()
        data[data.columns[x]] = data[data.columns[x]].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') 

### Muestro cuantos datos habia y cuantos quedaron luego de eliminar las filas duplicadas

In [11]:
antes_de_drop, despues_de_drop

(3800, 3544)

### Verifico las regiones y provincias

In [12]:
data.region_1.unique()

array(['cafayate', 'mendoza', 'salta', 'lujan de cuyo', 'san juan', '',
       'uco valley', 'perdriel', 'tupungato', 'valle de uco',
       'calchaqui valley', 'vista flores', 'maipu', 'patagonia',
       'alto valle del rio negro', 'la consulta', 'agrelo', 'neuquen',
       'rio negro valley', 'la rioja', 'famatina valley', 'argentina',
       'tulum valley', 'buenos aires', 'jujuy', 'san carlos',
       'pedernal valley', 'zonda valley', 'medrano', 'altos de mendoza',
       'san rafael', 'catamarca'], dtype=object)

In [13]:
data.province.unique()

array(['other', 'mendoza province'], dtype=object)

### Cambio la provincia mendoza province a mendoza

In [14]:
data.province = data.province.str.replace('mendoza province', 'mendoza')

### Cambio las regiones que estan en ingles a castellano

In [15]:
data.replace({'region_1' : { 'uco valley' : 'valle de uco', 'calchaqui valley' : 'valle calchaqui', 
                            'rio negro valley' : 'alto valle del rio negro', 
                            'famatina valley'  : 'valle de famatina', 'tulum valley'  : 'oasis del tulum',
                            'pedernal valley'  : 'valle de pedernal', 'zonda valley'  : 'oasis de zonda' }}, inplace = True)

### Me fijo que las regiones hayan quedado ok

In [16]:
data.region_1.unique()

array(['cafayate', 'mendoza', 'salta', 'lujan de cuyo', 'san juan', '',
       'valle de uco', 'perdriel', 'tupungato', 'valle calchaqui',
       'vista flores', 'maipu', 'patagonia', 'alto valle del rio negro',
       'la consulta', 'agrelo', 'neuquen', 'la rioja',
       'valle de famatina', 'argentina', 'oasis del tulum',
       'buenos aires', 'jujuy', 'san carlos', 'valle de pedernal',
       'oasis de zonda', 'medrano', 'altos de mendoza', 'san rafael',
       'catamarca'], dtype=object)

### Una de las regiones es Argentina <br> Usando la vineria puedo obtener de que provincias son por lo que armo dos listas con las provincias y vinerias unicas donde la region sea argentina <br> Luego con un for voy recorriendo y donde la region era Argentina lo reemplazo por la provincia donde esta la vineria

In [17]:
lista_provincia = ['cordoba','mendoza','mendoza','mendoza','neuquen','mendoza','mendoza','mendoza','mendoza','mendoza','catamarca','san juan','mendoza','mendoza','mendoza','mendoza','mendoza']
lista_vineria = ['bodega las canitas','haarth','terrazas de los andes','san huberto','familia schroeder','rutini','mendoza vineyards','aguijon de abeja','cave extreme','ruca malen','alto 3','finca las moras','casa de campo','battle axe','lost vineyards','staphyle','trapiche']

In [18]:
for x in range (0, len(lista_provincia)):
    for i in range(0, len(data)):
        if data.winery[i] == lista_vineria[x]:
            data.loc[i,'region_1'] = lista_provincia[x]

### Hay provincia llamada other por lo que al tener ya las regiones completas reemplazo ese other por la region que le corresponde

In [19]:
for x in range (0, len(data)):
    if data.province[x] == 'other':
        data.loc[x,'province'] = data.region_1[x]

### Hay algunas regiones vacias por lo que las reemplazo por la provincia que le corresponde

In [20]:
for x in range(0, len(data)):
    if data.region_1[x] == '':
        data.loc[x,'region_1'] = data.loc[x,'province']

### Hay provincias que estan con el dato de la region como por ejemplo Cafayate cuando tiene que ser Salta. <br> Ya que tengo las regiones completas elimino la columna provincia

In [21]:
data.drop('province', axis=1, inplace=True)

### Creo un mapeo donde detallo a que provincia corresponden las regiones que salen en las provincias

In [22]:
mapeo = {
    'cafayate' : 'salta',
    'lujan de cuyo'	: 'mendoza',
    'valle de uco' : 'mendoza',
    'perdriel' : 'mendoza',
    'tupungato'	: 'mendoza',
    'valle calchaqui' : 'valle calchaqui',
    'vista flores' : 'mendoza',
    'maipu'	: 'maipu',
    'patagonia'	: 'patagonia',
    'alto valle del rio negro' : 'rio negro',
    'la consulta' : 'mendoza',
    'agrelo' : 'mendoza',
    'valle de famatina'	: 'la rioja',
    'oasis del tulum' : 'san juan',
    'san carlos' : 'salta',
    'oasis de zonda' : 'san juan',
    'medrano' : 'mendoza',
    'valle de pedernal'	: 'san juan',
    'altos de mendoza' : 'mendoza',
    'san rafael' : 'mendoza',
    'mendoza' : 'mendoza',
    'salta' : 'salta',
    'san juan' : 'san juan',
    'neuquen' : 'neuquen',
    'la rioja' : 'la rioja',
    'cordoba' : 'cordoba',
    'jujuy' : 'jujuy',
    'catamarca' : 'catamarca'
}

### Y con ese diccionario hago un map creando la columna provincia

In [23]:
data['province'] = data['region_1'].map(mapeo)

### Ahora ya tengo las provincias y las regiones bien <br> Ej: Si es region Cafayate es provincia Salta

In [24]:
data.head(10)

,index,points,price,region_1,title,variety,winery,creacion,categoria,province
0,16,87,30.0,cafayate,felix lavaque 2010 felix malbec (cafayate),malbec,felix lavaque,2010,promotor,salta
1,17,87,13.0,mendoza,gaucho andino 2011 winemaker selection malbec ...,malbec,gaucho andino,2011,promotor,mendoza
2,183,88,12.0,salta,alamos 2007 torrontes (salta),torrontes,alamos,2007,promotor,salta
3,224,90,22.0,lujan de cuyo,mendel 2014 lunta malbec (lujan de cuyo),malbec,mendel,2014,promotor,mendoza
4,231,85,10.0,mendoza,andean sky 2007 bonarda (mendoza),bonarda,andean sky,2007,promotor,mendoza
5,245,85,12.0,san juan,finca las moras 2007 reserve chardonnay (san j...,chardonnay,finca las moras,2007,promotor,san juan
6,253,85,15.0,lujan de cuyo,cueva de las manos 2007 reserve malbec (lujan ...,malbec,cueva de las manos,2007,promotor,mendoza
7,261,89,37.0,mendoza,algodon 2008 estate blend gran reserva red,red blend,algodon,2008,promotor,mendoza
8,266,89,14.0,mendoza,kaiken 2008 corte malbec-bonarda-petit verdot ...,red blend,kaiken,2008,promotor,mendoza
9,273,89,19.0,valle de uco,o. fournier 2007 b crux red (uco valley),red blend,o. fournier,2007,promotor,mendoza


### Verifico las provincias unicas para ver si esta todo ok <br> Encuentro 3 provincias que hay que arreglar ya que no son provincias: Valle Calchaqui, Maipu, Patagonia

In [25]:
data.province.unique()

array(['salta', 'mendoza', 'san juan', 'valle calchaqui', 'maipu',
       'patagonia', 'rio negro', 'neuquen', 'la rioja', 'cordoba',
       'jujuy', 'catamarca'], dtype=object)

### Busco todas las vinerias de esas 3 provincias <br> Googleo de donde son (unas 42)

In [26]:
vineria_provincia = ['aniello','humberto canale','bodega chacra','bodega noemia de patagonia','bodega nqn','bodega patritti','denario','ruta 22','alpataco','bodega del fin del mundo','fabre montmayou','nemesio','restivo','schroeder estate','bodegas fabre','manos negras','southern edge','valle azul','nieto senetiner','infinitus','verum','aymara','bodega don bosco','pascual toso','luigi bosca','tercos','viamonte','coiron','palo alto','finca los olmos','zuccardi','el esteco','michel torino','colome','amalaya','inca','bodega tacuil']
provincia_vineria = ['rio negro','rio negro','rio negro','rio negro','neuquen','neuquen','rio negro','mendoza','neuquen','neuquen','mendoza','neuquen','mendoza','neuquen','mendoza','mendoza','neuquen','mendoza','mendoza','rio negro','rio negro','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','mendoza','salta','salta','salta','salta','salta','salta']

### Ahora recorriendo si la vineria es igual a la vineria_provincia lo que hago es reemplazar la provincia por la posicion que corresponda en provincia_vineria <br> Ej la posicion 0 es vineria aniello como es igual a la primer posicion de vineria_provincia a data.provincia en la posicion 0 la relleno con la provincia_vineria en al posicion 0 o sea: Rio Negro

In [27]:
for x in range (0, len(vineria_provincia)):
    for i in range(0, len(data)):
        if data.winery[i] == vineria_provincia[x]:
            data.loc[i,'province'] = provincia_vineria[x]

### Ahora al verificar las provincias vemos que esta todo ok

In [28]:
data.province.unique()

array(['salta', 'mendoza', 'san juan', 'rio negro', 'neuquen', 'la rioja',
       'cordoba', 'jujuy', 'catamarca'], dtype=object)

### Verifico los nulos

In [29]:
data.isnull().sum()

index         0
points        0
price        43
region_1      0
title         0
variety       0
winery        0
creacion      0
categoria     0
province      0
dtype: int64

### Completo todos los precios nulos con 0, al ser float los rellena con 0.0

In [30]:
data.price = data.price.fillna(value=0)

### Para obtener un precio a rellenar lo que hago es ir recorriendo todas las filas donde el precio sea igual a 0.0 y creo una variable donde van a estar todos los precios de los vinos segun variedad, provincia, region_1 y vineria y de eso obtengo la media y relleno el precio segun corresponda <br> Ej: Si la variedad es Malbec, la provincia es Mendoza, la region es Cafayate y la vineria es Felix Lavaque, la media del precio de todos los vinos que cumplan con esas condiciones va a ser el dato con el que voy a rellenar los price 0.0 donde todos los otros campos cumplan las mismas condiciones

In [31]:
for x in range(0, len(data)):
    if data.price[x] == 0.0:
        variedad = data[data.variety == data.variety[x]]
        variedad_provincia = variedad[variedad.province == data.province[x]]
        variedad_provincia_region = variedad_provincia[variedad_provincia.region_1 == data.region_1[x]]
        variedad_provincia_vineria = variedad_provincia.price[variedad_provincia.winery == data.winery[x]]
        data.loc[x,'price'] = variedad_provincia_vineria.median()

### Quedan 4 valores sueltos con valor 0.0 ya que no tienen un precio y no hay otros con esa vineria y variedad <br> Por lo que para llenar estos ultimos hago lo mismo que arriba solo que poniendo menos condiciones solo basandome en la provincia y la variedad <br> Por ejemplo si alguno de estos 4 es Malbec, Mendoza, cambo el 0.0 por la media de todos los vinos Malbec de Mendoza

In [32]:
for x in range(0, len(data)):
    if data.price[x] == 0.0:
        provincia = data[data.province == data.province[x]]
        provincia_variety = provincia.price[provincia.variety == data.variety[x]]
        data.loc[x,'price'] = provincia_variety.median()

### Ya no quedan nulos

In [33]:
data.isnull().sum()

index        0
points       0
price        0
region_1     0
title        0
variety      0
winery       0
creacion     0
categoria    0
province     0
dtype: int64

### Ya tengo el data set ordenado, con filas nuevas, datos completos y solo se eliminaron <span style="color: green">256</span> filas duplicadas no filas unicas que tengan valores NaN

In [34]:
antes_de_drop - despues_de_drop

256

In [35]:
data.rename(columns={'index':'id',
                    'points':'puntaje',
                    'price':'precio',
                    'region_1':'region',
                    'title':'titulo',
                    'variety':'variedad',
                    'winery':'bodega',
                    'province':'provincia'}, 
                     inplace=True)

In [36]:
data.head(10)

,id,puntaje,precio,region,titulo,variedad,bodega,creacion,categoria,provincia
0,16,87,30.0,cafayate,felix lavaque 2010 felix malbec (cafayate),malbec,felix lavaque,2010,promotor,salta
1,17,87,13.0,mendoza,gaucho andino 2011 winemaker selection malbec ...,malbec,gaucho andino,2011,promotor,mendoza
2,183,88,12.0,salta,alamos 2007 torrontes (salta),torrontes,alamos,2007,promotor,salta
3,224,90,22.0,lujan de cuyo,mendel 2014 lunta malbec (lujan de cuyo),malbec,mendel,2014,promotor,mendoza
4,231,85,10.0,mendoza,andean sky 2007 bonarda (mendoza),bonarda,andean sky,2007,promotor,mendoza
5,245,85,12.0,san juan,finca las moras 2007 reserve chardonnay (san j...,chardonnay,finca las moras,2007,promotor,san juan
6,253,85,15.0,lujan de cuyo,cueva de las manos 2007 reserve malbec (lujan ...,malbec,cueva de las manos,2007,promotor,mendoza
7,261,89,37.0,mendoza,algodon 2008 estate blend gran reserva red,red blend,algodon,2008,promotor,mendoza
8,266,89,14.0,mendoza,kaiken 2008 corte malbec-bonarda-petit verdot ...,red blend,kaiken,2008,promotor,mendoza
9,273,89,19.0,valle de uco,o. fournier 2007 b crux red (uco valley),red blend,o. fournier,2007,promotor,mendoza


In [38]:
data.sort_values(['puntaje', 'precio'], ascending=[False, False], inplace=True)

In [39]:
data.head(10)

,id,puntaje,precio,region,titulo,variedad,bodega,creacion,categoria,provincia
2278,82754,97,120.0,mendoza,bodega catena zapata 2006 nicasia vineyard mal...,malbec,bodega catena zapata,2006,promotor,mendoza
2980,109230,96,50.0,tupungato,riglos 2009 gran corte las divas vineyard red ...,bordeaux-style red blend,riglos,2009,promotor,mendoza
3000,109840,96,48.0,mendoza,alta vista 2013 single vineyard temis malbec (...,malbec,alta vista,2013,promotor,mendoza
3320,122027,95,225.0,mendoza,vina cobos 2010 cobos marchiori vineyard malbe...,malbec,vina cobos,2010,promotor,mendoza
2158,78302,95,100.0,mendoza,terrazas de los andes 2009 single parcel los c...,malbec,terrazas de los andes,2009,promotor,mendoza
2159,78303,95,90.0,salta,colome 2010 reserva malbec (salta),malbec,colome,2010,promotor,salta
281,8869,95,74.0,mendoza,pascual toso 2014 finca pedregal single vineya...,cabernet sauvignon-malbec,pascual toso,2014,promotor,mendoza
1143,38984,95,62.0,mendoza,finca perdriel 2007 vineyard selection red (me...,bordeaux-style red blend,finca perdriel,2007,promotor,mendoza
2415,88861,95,50.0,mendoza,trapiche 2009 finca jorge miralles single vine...,malbec,trapiche,2009,promotor,mendoza
3445,126237,95,50.0,mendoza,vistalba 2007 corte a red (mendoza),red blend,vistalba,2007,promotor,mendoza
